# Use Case 1

Find the companies where X has worked, and their roles at those companies

## What questions would we have to ask of our data?

_"Which companies has X worked for, and in what roles?"_

Reviewing this question we can identify several entities, attributes and relationships. We have the concept of a _company_, a _person_ (X), and a _role_. Further, a person _worked for_ a company.

_Company_ and _Person_ are both entities, which we'll model as vertices with appropriate labels. For now, we'll assume a direct relationship between a person and a company: a person _WORKED FOR_ a company. We'll make _role_ an attribute of this relationship.

Adding in a few properties – _firstName_ and _lastName_ for a person, _name_ for a company – we end up with the following data model:

<img src="https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-sagemaker/images/data-modelling-04.png"/>

### Keep it simple

Over the course of this exercise we'll see _role_ change place several times. At this stage it's a simple attribute of a relationship. In later steps we'll see it promoted to being a vertex in its own right.

As far as our current use case is concerned, role appears to be a simple value type, much like colour, height or weight. If it were a complex value type with several fields – such as address – or if there were some explicit structural relations between values – as there are in a category hierarchy – we would consider making it a vertex from the outset.

## Sample dataset

We'll now create a sample dataset in line with our model. We'll include enough data to ensure that our queries have to exclude some portions of the graph in order to return a correct result.

<img src="https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-sagemaker/images/data-modelling-01.png"/>

### Creating our sample data

In [3]:
%load_ext ipython_unittest
%run '../../../neptune-sagemaker/notebooks/util/neptune.py'

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


In [4]:
neptune.clear()
g = neptune.graphTraversal()

clearing data...
clearing property graph data [edge_batch_size=200, edge_count=Unknown]...
clearing property graph data [edge_batch_size=200, edge_count=56958]...
clearing property graph data [edge_batch_size=200, edge_count=56758]...
clearing property graph data [edge_batch_size=200, edge_count=56558]...
clearing property graph data [edge_batch_size=200, edge_count=56358]...
clearing property graph data [edge_batch_size=200, edge_count=56158]...
clearing property graph data [edge_batch_size=200, edge_count=55958]...
clearing property graph data [edge_batch_size=200, edge_count=55758]...
clearing property graph data [edge_batch_size=200, edge_count=55558]...
clearing property graph data [edge_batch_size=200, edge_count=55358]...
clearing property graph data [edge_batch_size=200, edge_count=55158]...
clearing property graph data [edge_batch_size=200, edge_count=54958]...
clearing property graph data [edge_batch_size=200, edge_count=54758]...
clearing property graph data [edge_batch_size

In [5]:
(g.
   addV('Person').property(id,'p-1').property('firstName','Martha').property('lastName','Rivera').
   addV('Person').property(id,'p-2').property('firstName','Richard').property('lastName','Roe').
   addV('Person').property(id,'p-3').property('firstName','Li').property('lastName','Juan').
   addV('Person').property(id,'p-4').property('firstName','John').property('lastName','Stiles').
   addV('Person').property(id,'p-5').property('firstName','Saanvi').property('lastName','Sarkar').
   addV('Company').property(id,'c-1').property('name','Example Corp').
   addV('Company').property(id,'c-2').property('name','AnyCompany').
   V('p-1').addE('WORKED_FOR').to(V('c-1')).property('role','Principal Analyst').                         
   V('p-2').addE('WORKED_FOR').to(V('c-1')).property('role','Senior Analyst').                           
   V('p-3').addE('WORKED_FOR').to(V('c-1')).property('role','Analyst').
   V('p-4').addE('WORKED_FOR').to(V('c-1')).property('role','Analyst').                           
   V('p-5').addE('WORKED_FOR').to(V('c-2')).property('role','Manager').
   V('p-3').addE('WORKED_FOR').to(V('c-2')).property('role','Associate Analyst').
   toList())

[e[78b71093-0d41-c6f7-8d49-9dba479bfb38][p-3-WORKED_FOR->c-2]]

## Querying the data

### Query 1 – Which companies has Li worked for, and in what roles?

To answer this question, we'll have to perform the following steps:

 1. Start at the Person vertex representing Li
 2. Follow WORKED_FOR edges to find each Company for whom Li has worked
 3. Select the Company details, and the role property of the relationship

### Write a failing unit test

In [6]:
%%unittest

results = None # TODO

assert results == [{'company': 'Example Corp', 'role': 'Analyst'}, 
                   {'company': 'AnyCompany', 'role': 'Associate Analyst'}]

Fail

F
FAIL: test_1 (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 5, in test_1
AssertionError: None != [{'company': 'Example Corp', 'role': 'Ana[58 chars]st'}]

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

### And write the query to make it pass

In [7]:
%%unittest

results = (g.V('p-3').
             outE('WORKED_FOR').as_('e').
             otherV().
             project('company', 'role').
             by('name').
             by(select('e').values('role')).
             toList())

assert results == [{'company': 'Example Corp', 'role': 'Analyst'}, 
                   {'company': 'AnyCompany', 'role': 'Associate Analyst'}]

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.077s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>